In [23]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import joblib
import time
import os

from gtts import gTTS
from playsound import playsound

In [24]:
import os
print(os.getcwd())


c:\Users\Acer\OneDrive\Desktop\Signaura project


In [25]:
import os

path = "model.keras"
print(os.path.exists(path))



True


In [26]:
model = tf.keras.models.load_model("model.keras")

In [27]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               8192      
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 381)               12573     
                                                                 
Total params: 31,101
Trainable params: 31,101
Non-trai

In [28]:
with open("label_encoder_382.pkl", "rb") as f:
    le = pickle.load(f)


In [29]:
# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)
mp_draw = mp.solutions.drawing_utils

In [31]:
# -----------------------------
# Keypoint importance function
# -----------------------------
def compute_keypoint_importance(model, row, pred_index=None):
    row = np.array(row).reshape(1, 63)
    x = tf.convert_to_tensor(row, dtype=tf.float32)
    
    with tf.GradientTape() as tape:
        tape.watch(x)
        preds = model(x, training=False)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        pred_score = preds[0, pred_index]
    
    grads = tape.gradient(pred_score, x).numpy()[0]
    
    importance = np.zeros(21)
    for i in range(21):
        importance[i] = np.sum(np.abs(grads[i*3:(i+1)*3]))
    
    importance = importance / (np.max(importance) + 1e-8)
    return importance

# -----------------------------
# Draw heatmap on hand
# -----------------------------
def draw_hand_heatmap(frame, hand_landmarks, importance):
    for i, lm in enumerate(hand_landmarks):
        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
        score = importance[i]
        if score > 0.66:
            color = (0, 0, 255)  # Red = most important
        elif score > 0.33:
            color = (0, 255, 255)  # Yellow = medium
        else:
            color = (255, 0, 0)  # Blue = least
        cv2.circle(frame, (x, y), 8, color, -1)

# -----------------------------
# Start webcam
# -----------------------------
cap = cv2.VideoCapture(0)

en_sentense = []
sh_sentense = []
tm_sentense = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        lm = result.multi_hand_landmarks[0]
        row = []

        # Extract landmarks
        for l in lm.landmark:
            row.extend([l.x, l.y, l.z])

        # Draw basic landmarks
        mp_draw.draw_landmarks(frame, lm, mp_hands.HAND_CONNECTIONS)

        # Reshape for model
        row = np.array(row).reshape(1, 63)

        # Predict
        pred = model.predict(row, verbose=0)[0]
        pred_index = np.argmax(pred)
        prob = np.max(pred)

        if prob > 0.6:
            predicted_class = le.inverse_transform([pred_index])[0]
            parts = predicted_class.split("|")
            english = parts[0]
            sinhala = parts[1] if len(parts) > 1 else ""
            tamil = parts[2] if len(parts) > 2 else ""

            en_sentense.append(english)
            sh_sentense.append(sinhala)
            tm_sentense.append(tamil)

            # Display text
            cv2.putText(frame, f"English: {english}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Sinhala: {sinhala}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Tamil: {tamil}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Print in console
            # print(f"English: {english} | Sinhala: {sinhala} | Tamil: {tamil}")

            print(" ".join(en_sentense))
            print(" ".join(sh_sentense))
            print(" ".join(tm_sentense))

            # Compute and draw heatmap
            importance = compute_keypoint_importance(model, row, pred_index)
            draw_hand_heatmap(frame, lm.landmark, importance)
        else:
            cv2.putText(frame, "Uncertain Prediction", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    else:
        cv2.putText(frame, "No Hand Detected", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show frame
    cv2.imshow("SLSL Recognition with Heatmap", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Can't
බැහැ
முடியாது
Can't Can't
බැහැ බැහැ
முடியாது முடியாது
Can't Can't Wash
බැහැ බැහැ සෝදන්න
முடியாது முடியாது கழுவு


c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Can't Can't Wash Wash
බැහැ බැහැ සෝදන්න සෝදන්න
முடியாது முடியாது கழுவு கழுவு
Can't Can't Wash Wash Wash
බැහැ බැහැ සෝදන්න සෝදන්න සෝදන්න
முடியாது முடியாது கழுவு கழுவு கழுவு


c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetP

Can't Can't Wash Wash Wash Wash Wash Wash Wash Wash Wash Wash Wash Structure Good night Good night Good night Good night Good night Good night Good night Good night Good night Good night Deep Deep Deep Deep Deep Saree Saree List List List Saree Wrong Wrong Wrong Wrong Wrong Wrong Wrong Wrong Deep Can't Wash Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Saree Saree Can't Saree Saree Saree Saree Saree Saree Can't Can't Can't Can't Can't Structure Structure Structure Structure Structure Structure Structure Structure Structure Structure Structure Out Out Out Wash
බැහැ බැහැ සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න ව්‍යුහය සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් ගැඹුරු ගැඹුරු ගැඹුරු ගැඹුරු ගැඹුරු සාරි සාරි ලැයිස්තුව ලැයිස්තුව ලැයිස්තුව සාරි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි ගැඹුරු බැහැ සෝදන්න බැහැ බැහැ 

c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Can't Can't Wash Wash Wash Wash Wash Wash Wash Wash Wash Wash Wash Structure Good night Good night Good night Good night Good night Good night Good night Good night Good night Good night Deep Deep Deep Deep Deep Saree Saree List List List Saree Wrong Wrong Wrong Wrong Wrong Wrong Wrong Wrong Deep Can't Wash Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Can't Saree Saree Can't Saree Saree Saree Saree Saree Saree Can't Can't Can't Can't Can't Structure Structure Structure Structure Structure Structure Structure Structure Structure Structure Structure Out Out Out Wash Can't
බැහැ බැහැ සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න සෝදන්න ව්‍යුහය සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් සුබ රාත්‍රියක් ගැඹුරු ගැඹුරු ගැඹුරු ගැඹුරු ගැඹුරු සාරි සාරි ලැයිස්තුව ලැයිස්තුව ලැයිස්තුව සාරි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි වැරදි ගැඹුරු බැහැ සෝදන්න බැහැ

c:\Users\Acer\OneDrive\Desktop\Signaura project\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
